## Création mail Yopmail

In [1]:
import yopmail

email = yopmail.create_random_email()

if email['status'] == 'success':
    generated_email = email['generated_email']
    print(generated_email)
else:
    print(email['error_message'])
    print(email['execution_time'])

grojatemmomi-2696@yopmail.com


## Connexion Hightest

## Lecture mail Yopmail

In [2]:
# report = yopmail.last_message_of(email['identifier_email'])

report = yopmail.last_message_of('zeugreretega-3236')

if report['status'] == 'success':
    content_report = report['content']
    print(content_report)
else:
    print(report['error_message'])
    print(report['execution_time'])

Corp de mail 
seconde ligne de mail 
troisième ligne de mail 
truc bidule 
